# DataEngineeringExercise

In [1]:
import pandas as pd
import requests

### Importing the data into notebook

In [5]:
fname='/Users/froux/DataEngineeringExercise/patients.csv'

In [8]:
pat2drug = pd.read_csv(
    fname,
    sep=','
)

In [10]:
print(pat2drug.shape)

(248969, 2)


In [413]:
# number of unique NDCs
len(pat2drug['ndc_number'].unique())

11223

In [414]:
# number of unique patient IDs
len(pat2drug['patient_id'].unique())

19999

In [11]:
display(pat2drug.head(10))

,patient_id,ndc_number
0,1,527134301
1,1,378180510
2,1,68180075103
3,1,64380080306
4,1,49281040365
5,1,115552202
6,1,68180075009
7,1,781279001
8,1,68180046801
9,1,69097082507


In [ ]:
any(pat2drug['ndc_number'] == 527134301)

In [40]:
pat2drug['ndc_number'].astype(str).apply(lambda x: len(x)).unique()

array([ 9, 11,  8,  7, 10,  1])

In [699]:
len(pat2drug['patient_id'].unique())

19999

### Translate NDCs between 10- and 11-digit formats

In [338]:
def convert_ndc(ndc_number,ndigits):
    
    ndc_number = str(ndc_number)
    ndc_out= None
    
    if (ndigits==11):
        if (len(ndc_number)<11):
            if (len(ndc_number)<10):
                prefix = (10-len(ndc_number))*'0'
                ndc_number = f'''{prefix}{ndc_number}'''
        
            if (ndc_number[0:3].find('0')==0):
                ndc_out=f'''0{ndc_number[0:4]}-{ndc_number[4:8]}-{ndc_number[8:11]}'''
            elif (ndc_number[5:8].find('0')==0):
                ndc_out=f'''{ndc_number[0:5]}-0{ndc_number[5:8]}-{ndc_number[8:11]}'''  
            else:
                ndc_out=f'''{ndc_number[0:5]}-{ndc_number[5:9]}-0{ndc_number[9:11]}'''
        else:
            ndc_out=f'''{ndc_number[0:5]}-{ndc_number[5:9]}-{ndc_number[9:11]}'''
        

    if (ndigits==10):
        
        if (ndc_number.find('-')>-1):
            raise Exception("Unexpected format detected")
        if (ndc_number[0:1].find('0')==0):
            ndc_out=f'''{ndc_number[1:5]}-{ndc_number[5:9]}-{ndc_number[9:11]}'''
        elif (ndc_number[5:6].find('0')==0):
            ndc_out=f'''{ndc_number[0:5]}-{ndc_number[6:9]}-{ndc_number[9:11]}'''
        else:
            ndc_out=f'''{ndc_number[0:5]}-{ndc_number[5:9]}-{ndc_number[10:11]}'''

    if (len(ndc_out)-2!=ndigits):
        raise Exception(f'"Output length {len(ndc_out)-2} does not match expected value {ndigits}"')
    
    return(ndc_out)

In [339]:
# unit testing using 11-digit format
all([
    convert_ndc(2759701,11) =='00002-7597-01',
    convert_ndc(5024204062,11) =='50242-0040-62',
    convert_ndc(6057541121,11) =='60575-4112-01',
    convert_ndc(90999999999,11) =='90999-9999-99'
])

True

In [340]:
# unit testing using 10-digit format
all([
    convert_ndc('00002759701',10)=='0002-7597-01',
    convert_ndc('60575411201',10)=='60575-4112-1',
    convert_ndc('50242004062',10)=='50242-040-62']
)

True

### Convert NDCs into 11- and 10-digit format values

In [341]:
# first convert into 11-digit format
pat2drug['ndc_number_11'] = pat2drug['ndc_number'].apply(convert_ndc,args=[11])
# then use 11-format to convert to 10-format
pat2drug['ndc_number_10'] = pat2drug['ndc_number_11'].str.replace('-','').apply(convert_ndc,args=[10])

In [342]:
display(pat2drug.head(5))

,patient_id,ndc_number,ndc_number_11,ndc_number_10
0,1,527134301,00527-1343-01,0527-1343-01
1,1,378180510,00378-1805-10,0378-1805-10
2,1,68180075103,68180-0751-03,68180-751-03
3,1,64380080306,64380-0803-06,64380-803-06
4,1,49281040365,49281-0403-65,49281-403-65


### 1. How many patients have been prescribed a medication containing the active ingredient LEVODOPA?

In [393]:
base_url="https://api.fda.gov/drug/ndc.json"

In [394]:
search_url="?search=active_ingredients.name:\"LEVODOPA\"&limit=1000"

In [395]:
query = f'''{base_url}{search_url}'''

In [396]:
print(query)

https://api.fda.gov/drug/ndc.json?search=active_ingredients.name:"LEVODOPA"&limit=1000


In [397]:
response = requests.get(query)

In [398]:
leva_ndc = pd.DataFrame(
    [pack['package_ndc'] for rec1 in response.json()['results'] for pack in rec1['packaging']],
    columns=['ndc_number']
)

In [399]:
# all NDCs for levadopa are in 10-format
leva_ndc['ndc_number'].str.replace('-','').apply(len).unique()

array([10])

In [400]:
# accordingly we convert levadopa NDCs to 11-format
leva_ndc['ndc_number_11'] = leva_ndc['ndc_number'].str.replace('-','').astype(int).apply(convert_ndc,args=[11])

In [401]:
leva_ndc.head(5)

,ndc_number,ndc_number_11
0,51407-168-01,51407-1680-01
1,51407-168-05,51407-1680-05
2,0781-5625-01,00781-5625-01
3,63629-2357-1,63629-2357-01
4,51407-166-01,51407-1660-01


In [403]:
# return number of patients using levadopa 
pat2drug[pat2drug['ndc_number_11'].isin(leva_ndc['ndc_number_11'])].shape[0]

705

In [405]:
# should be same as above
pat2drug.merge(
    leva_ndc,
    on=['ndc_number_11']
).shape[0]

705

In [649]:
print(f"""The number of patients prescribed with LEVODOPA active ingredient is {pat2drug[pat2drug['ndc_number_11'].isin(leva_ndc['ndc_number_11'])].shape[0]}""")

The number of patients prescribed with LEVODOPA active ingredient is 705


### 2. How many of the ndc numbers/drug packages can't be found in FDA's database?

In [412]:
# inspect NDCs
pat2drug['ndc_number_10'].sort_values()

25881     0000-0000-00
6973      0000-0000-00
20044     0000-0000-00
223409    0000-0000-00
59390     0000-0000-00
              ...     
91252     99073-708-22
246117    99073-708-22
246590    99073-708-27
248763    99207-464-30
246722    99999-3340-1
Name: ndc_number_10, Length: 248969, dtype: object

In [423]:
# helper function to build queries for each NDC
def build_query(ndc_nr):
    base_url="https://api.fda.gov/drug/ndc.json"
    search_url=f"?search=packaging.package_ndc:\"{ndc_nr}\"&limit=1"
    query = f'''{base_url}{search_url}'''
    return(query)

In [474]:
# make a test
query = build_query('51407-168-01')

In [475]:
print(query)

https://api.fda.gov/drug/ndc.json?search=packaging.package_ndc:"51407-168-01"&limit=1


In [429]:
ndc2fda = pd.DataFrame(
    [
        (ndc,requests.get(q).status_code==200) for ndc,q in zip(pd.Series(pat2drug['ndc_number_10'].unique()),pd.Series(pat2drug['ndc_number_10'].unique()).apply(build_query))
    ],
    columns=['ndc_number_10','fda_record']
)

In [430]:
display(ndc2fda.head(5))

,ndc_number_10,fda_record
0,0527-1343-01,False
1,0378-1805-10,True
2,68180-751-03,False
3,64380-803-06,True
4,49281-403-65,False


In [432]:
print(ndc2fda.shape[0])

# number of NDCs found in fda´s db
print(ndc2fda[ndc2fda['fda_record']].shape[0])

11222
8765


##### Approach above is very slow due to large number of single queries executed, but brings ground truth! Code below shows optimization by adding more NDCs inside each query step.

In [633]:
unique_ndcs = pd.DataFrame(pat2drug['ndc_number_10'].unique(),columns=['ndc_10_pat'])

In [634]:
idx = list(range(0,unique_ndcs.shape[0],500))
idx.append(unique_ndcs.shape[0])
for i in range(0,len(idx)-1):
    print(idx[i],idx[i+1])

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
6500 7000
7000 7500
7500 8000
8000 8500
8500 9000
9000 9500
9500 10000
10000 10500
10500 11000
11000 11222


In [645]:
base_url="https://api.fda.gov/drug/ndc.json"

ndc2fda2 = pd.DataFrame([],columns=['ndc_10_pat','ndc_10_fda'])
for i in range(0,len(idx)-1):
    search_url="?search=packaging.package_ndc:("+"+OR+".join(unique_ndcs['ndc_10_pat'][idx[i]:idx[i+1]])+")&limit=1000"
    query = f'''{base_url}{search_url}'''
    response = requests.get(query)
    if (response.status_code==200):
        fda_ndc = pd.DataFrame(
            [pack['package_ndc'] for rec1 in response.json()['results'] for pack in rec1['packaging']],
            columns=['ndc_10_fda'])
        if (ndc2fda2.shape[0]<1):
            ndc2fda2 = unique_ndcs[idx[i]:idx[i+1]].merge(
                            fda_ndc,
                            left_on=['ndc_10_pat'],
                            right_on=['ndc_10_fda']
                        ) #.drop_duplicates().sort_values(by='ndc_10_pat').reset_index(drop=True)
        else:
            ndc2fda2 = ndc2fda2.append(
                unique_ndcs[idx[i]:idx[i+1]].merge(
                    fda_ndc,
                    left_on=['ndc_10_pat'],
                    right_on=['ndc_10_fda']
                ),
                ignore_index=True #.drop_duplicates().sort_values(by='ndc_10_pat').reset_index(drop=True)
            )
            

In [647]:
display(ndc2fda2.head(5))
print(ndc2fda2.shape[0])

,ndc_10_pat,ndc_10_fda
0,0378-1805-10,0378-1805-10
1,64380-803-06,64380-803-06
2,0115-5522-02,0115-5522-02
3,0781-2790-01,0781-2790-01
4,68180-468-01,68180-468-01


8765


In [650]:
print(f"""The number of ndc numbers/drug packages that can't be found in FDA's database is {unique_ndcs.shape[0]-ndc2fda2.shape[0]}""")

The number of ndc numbers/drug packages that can't be found in FDA's database is 2457


### 3. Which ten active ingredients are prescribed to most patients and how many patients have been prescribed each of them?

In [670]:
def ingre2ndc(json_dat):
    df = pd.DataFrame(
        [(ing['name'],pack['package_ndc']) for rec in json_dat['results'] if 'active_ingredients' in rec for ing,pack in zip(rec['active_ingredients'],rec['packaging']) ],
        columns=['active_ingredient','ndc_10_fda']
    )
    return(df)

In [671]:
base_url="https://api.fda.gov/drug/ndc.json"

actingre2ndc = pd.DataFrame([],columns=['ndc_10_pat','ndc_10_fda'])
for i in range(0,len(idx)-1):
    search_url="?search=packaging.package_ndc:("+"+OR+".join(unique_ndcs['ndc_10_pat'][idx[i]:idx[i+1]])+")&limit=1000"
    query = f'''{base_url}{search_url}'''
    response = requests.get(query)
    if (response.status_code==200):
        fda_ndc = ingre2ndc(response.json())
        if (ndc2fda2.shape[0]<1):
            actingre2ndc = unique_ndcs[idx[i]:idx[i+1]].merge(
                            fda_ndc,
                            left_on=['ndc_10_pat'],
                            right_on=['ndc_10_fda']
                        ) #.drop_duplicates().sort_values(by='ndc_10_pat').reset_index(drop=True)
        else:
            actingre2ndc = actingre2ndc.append(
                unique_ndcs[idx[i]:idx[i+1]].merge(
                    fda_ndc,
                    left_on=['ndc_10_pat'],
                    right_on=['ndc_10_fda']
                ),
                ignore_index=True #.drop_duplicates().sort_values(by='ndc_10_pat').reset_index(drop=True)
            )
            

In [672]:
display(actingre2ndc.head(5))
print(actingre2ndc.shape[0])

,ndc_10_pat,ndc_10_fda,active_ingredient
0,0378-1805-10,0378-1805-10,LEVOTHYROXINE SODIUM
1,0781-2790-01,0781-2790-01,OMEPRAZOLE
2,68180-468-01,68180-468-01,LOVASTATIN
3,68382-760-01,68382-760-01,METFORMIN HYDROCHLORIDE
4,57237-233-05,57237-233-05,TRIMETHOPRIM


4938


In [690]:
top10=[i[0] for i in list(pat2drug.merge(
    actingre2ndc,
    left_on=['ndc_number_10'],
    right_on=['ndc_10_pat']
)[['active_ingredient']].value_counts().sort_values(ascending=False)[0:10].index)]

In [695]:
pat2ingr = pat2drug.merge(
    actingre2ndc,
    left_on=['ndc_number_10'],
    right_on=['ndc_10_pat']
)[['patient_id','active_ingredient']]

In [712]:
display(pat2ingr.head(5))

,patient_id,active_ingredient
0,1,LEVOTHYROXINE SODIUM
1,69,LEVOTHYROXINE SODIUM
2,246,LEVOTHYROXINE SODIUM
3,266,LEVOTHYROXINE SODIUM
4,297,LEVOTHYROXINE SODIUM


In [691]:
print(f"""The ten active ingredients prescribed to most patients are {top10}""")

The ten active ingredients prescribed to most patients are ['AMLODIPINE BESYLATE', 'ACETAMINOPHEN', 'AMOXICILLIN', 'LEVOTHYROXINE SODIUM', 'ALBUTEROL SULFATE', 'CIPROFLOXACIN HYDROCHLORIDE', 'FLUTICASONE PROPIONATE', 'APIXABAN', 'HYDROCHLOROTHIAZIDE', 'LISINOPRIL']


In [726]:
# The table below shows how many times each one of the top 10 ingredients has been prescribed:
pat2ingr['active_ingredient'][pat2ingr['active_ingredient'].isin(top10)].value_counts().sort_values(ascending=False)

AMLODIPINE BESYLATE            2657
ACETAMINOPHEN                  2318
AMOXICILLIN                    2272
LEVOTHYROXINE SODIUM           2202
ALBUTEROL SULFATE              2027
CIPROFLOXACIN HYDROCHLORIDE    1794
FLUTICASONE PROPIONATE         1665
APIXABAN                       1542
HYDROCHLOROTHIAZIDE            1463
LISINOPRIL                     1339
Name: active_ingredient, dtype: int64

### 4. Which ten drugs that are not found in FDA's database are prescribed to highest number of patients, and how many patients have been prescribed each of them?